In [ ]:
from flask import Flask, render_template
import requests
from bs4 import BeautifulSoup as bs
import schedule
import time
from threading import Thread

app = Flask(__name__)

collected_news = []  # 수집된 뉴스를 저장할 전역 변수

def save_news_to_file(news_list):
    """수집된 뉴스를 텍스트 파일로 저장"""
    with open("collected_news.txt", "a", encoding="utf-8") as f:
        for news in news_list:
            f.write(f"Title: {news['title']}\n")
            f.write(f"Date: {news['date']}\n")
            f.write(f"Content:\n{news['content']}\n")
            f.write("="*80 + "\n")  # 구분선 추가

def get_news(url: str):
    res = requests.get(url)
    soup = bs(res.text, "lxml")
    
    title = soup.select("h3.tit_view")[0].text
    reg_date = soup.select("span.num_date")[0].text
    content_list = soup.select("div.article_view p")
    
    content = ""
    for i in content_list:
        content += i.text
    
    return {
        "title": title,
        "date": reg_date,
        "content": content
    }

def collect_news():
    global collected_news  # 전역 변수에 접근
    news_list = []
    page = 1
    
    while page <= 3:  # 1~3페이지 수집
        url = f"https://news.daum.net/breakingnews/digital?page={page}"
        res = requests.get(url)
        soup = bs(res.text, "lxml")
        url_list = soup.select("ul.list_news2 a.link_txt")
        
        if len(url_list) == 0:
            break
        
        for news_url in url_list:
            news = get_news(news_url["href"])
            news_list.append(news)

        page += 1

    collected_news = news_list  # 수집된 뉴스를 전역 변수에 저장
    save_news_to_file(collected_news)  # 수집된 뉴스 파일에 저장

def scheduled_collect_news():
    """매일 자정에 뉴스 수집"""
    print("뉴스 수집 시작")
    collect_news()
    print("뉴스 수집 완료")

# 스케줄링 설정: 매일 16시에 뉴스 수집 실행
schedule.every().day.at("16:00").do(scheduled_collect_news)

def run_scheduler():
    """스케줄러를 실행하는 함수"""
    while True:
        schedule.run_pending()
        time.sleep(1)

# 스케줄러를 별도의 스레드에서 실행
thread = Thread(target=run_scheduler)
thread.start()

@app.route('/')
def index():
    return render_template('index.html', news_list=collected_news)

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8083)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8083
 * Running on http://192.168.20.117:8083
Press CTRL+C to quit
192.168.20.117 - - [04/Oct/2024 15:58:56] "GET / HTTP/1.1" 200 -


뉴스 수집 시작
뉴스 수집 시작
뉴스 수집 시작


192.168.20.117 - - [04/Oct/2024 16:00:06] "GET / HTTP/1.1" 200 -


뉴스 수집 완료
뉴스 수집 시작
뉴스 수집 완료
뉴스 수집 시작
뉴스 수집 완료
뉴스 수집 시작


192.168.20.117 - - [04/Oct/2024 16:00:09] "GET / HTTP/1.1" 200 -


뉴스 수집 완료
뉴스 수집 시작
뉴스 수집 완료
뉴스 수집 시작
뉴스 수집 완료
뉴스 수집 시작
뉴스 수집 완료
뉴스 수집 완료
뉴스 수집 완료


In [9]:
pip install schedule

Note: you may need to restart the kernel to use updated packages.
